### Beispiel 3: SIR-Modell für Epidemien


#### Variante a

Das SIR-Modell ist ein einfaches Modell zur Ausbreitung einer Epidemie. Es unterteilt die Bevölkerung in drei Gruppen: 
Empfängliche (Susceptible), Infizierte (Infected) und Genesene (Recovered) und modelliert die folgenden Effekte:
* Die Zahl der empfänglichen Personen nimmt durch Infektionen ab ($S \leadsto I$)
* Die Zahl der infizierten Personen nimmt durch Infektionen zu ($S \leadsto I$) und durch Genesungen ab ($I \leadsto R$)
* Die Zahl der genesenen Personen nimmt durch Genesungen zu ($I \leadsto R$)

Die beiden Prozesse sind Infektion und Genesung:
* Infektion:
    * Je mehr Infizierte es gibt, desto wahrscheinlicher wird eine empfängliche Person infiziert. 
    * Proportional zu $S(t)$ und $I(t)$, aber skaliert mit der Gesamtbevölkerung $N(t)=S(t)+I(t)+R(t)$
    * $\leadsto \beta \frac{I(t)}{N(t)}S(t)$ mit $\beta$ als Konstante.
* Genesung:
    * Je mehr Infizierte es gibt, desto mehr Genesungen treten auf.
    * $\leadsto \gamma I(t)$ mit $\gamma$ als Konstante.

Wir erhalten die Differentialgleichungen (ODEs):

\begin{align*}
\frac{dS}{dt}(t) &= - \beta \frac{I(t)}{N(t)} S(t) \\
\frac{dI}{dt}(t) &= \beta \frac{I(t)}{N(t)} S(t) - \gamma I(t) \\
\frac{dR}{dt}(t) &= \gamma I(t)
\end{align*}

In der gegebenen Form ist die unbekannte Funktion $y(t) = (S(t), I(t), R(t))$ und wir haben
\begin{align*}
\frac{dy}{dt}(t) &= f(t, y(t)), f(t, (S, I, R)) = (- \beta S I/N, \beta S I/N - \gamma I, \gamma I)
\end{align*}

In [ ]:
from numpy import array
import matplotlib.pyplot as plt

Definieren Sie das SIR-Modell durch die Funktion $f(t, y(t))$ und die entsprechenden Parameter $\beta$ und $\gamma$:

In [ ]:
class SIR_Modell:
    def __init__(self, beta, gamma):
        self.beta = beta
        self.gamma = gamma

    def f(self, t, y):
        S, I, R = y
        N = S + I + R
        return array([-self.beta*S*I/N, self.beta*S*I/N-self.gamma*I, self.gamma*I])

In [ ]:
from ipynb.fs.full.numode import EulerStep
from ipynb.fs.full.numode2 import HeunStep
from numpy import array

In [ ]:
def SimuliereSIR(method = EulerStep, model = SIR_Modell(1,1), start_t=0, start_y0=array([0.9,0.1,0]), dt=2):
    plt.xlabel('t')
    t = start_t
    y = start_y0.copy()
    t_values = [t]
    s_values = [y[0]]
    i_values = [y[1]]
    r_values = [y[2]]
    while t < 100+1e-12:
        y = method(model.f, t, y, dt)
        t += dt
        t_values.append(t)
        s_values.append(y[0])
        i_values.append(y[1])
        r_values.append(y[2])
    n_values = array(s_values)+array(i_values)+array(r_values)
    plt.plot(t_values,s_values,"-o")
    plt.plot(t_values,i_values,"-o")
    plt.plot(t_values,r_values,"-o")
    plt.plot(t_values,n_values,"-o")
    plt.legend(["S","I","R","N"])
    plt.show()

In [ ]:
sirm = SIR_Modell(beta = 1, gamma = 0.1)
y0 = array([0.8,0.1,0.1])
SimuliereSIR(method=EulerStep, model=sirm, start_y0 = y0, dt=0.2)
#SimuliereSIR(method=HeunStep, model=sirm, start_y0 = y0, dt=4)

#### Variante b

Geburten- und Sterberaten berücksichtigen


\begin{align*}
\frac{dS}{dt}(t) &= \nu N(t) - \beta \frac{I(t)}{N(t)} S(t) - \mu S(t) \\
\frac{dI}{dt}(t) &= \beta \frac{I(t)}{N(t)} S(t) - \gamma I(t) - \mu I(t) \\
\frac{dR}{dt}(t) &= \gamma I(t) - \mu R(t)
\end{align*}


### Aufgabe:
Erweitern Sie das SIR-Modell um Geburten- und Sterberaten und implementieren Sie das Modell.

In [ ]:
class Erweitertes_SIR_Modell:
    def __init__(self, beta, gamma, mu, nu):
        self.beta = beta
        self.gamma = gamma
        self.mu = mu
        self.nu = nu
    def f(self, t, y):
        S, I, R = y
        N = S+I+R
        return array([self.nu * N - self.beta * I/N*S-self.mu *S,
                      self.beta * I/N*S - self.gamma * I - self.mu * I,
                      self.gamma*I - self.mu*R])

In [ ]:
esirm = Erweitertes_SIR_Modell(beta = 1, gamma = 0.1, mu = 0.025, nu = 0.025)
y0 = array([0.8,0.1,0.1])
#SimuliereSIR(method=EulerStep, model=esirm, start_y0 = y0, dt=2)
SimuliereSIR(method=HeunStep, model=esirm, start_y0 = y0, dt=0.2)